
<div align="center">

## Bruteforce Concept Demonstration
### A demonstration of the concept of bruteforcing a website or email login written in Python.
[![Python Badge](https://img.shields.io/badge/Made_with_Python-3776AB?logo=python&labelColor=white)](https://python.org/)
##

[Intro](#introduction) • [Run it](#how-to-run-it) • [Use it](#how-to-use-it) • [Disclaimer](#disclaimer)

</div>

### Introduction
This project was assigned as a pet project for work. It is not meant to be used as a practical tool, but as an educational demonstration.

This tries to demonstrate the underlying code behind bruteforce network attacks, both to websites and emails.

### How to Run it
To run the application, simply invoke it with the appropriate flags for it to function. Using the `--help` flag will show a list of the available flags and options. Any missing flags for the code to function will be indicated on the terminal window.

###### Run it with python:
```sh
python3 bruteforcer.py --help
```

###### Or run it directly with a shell:
```sh
$(pwd)/bruteforcer.py --help
```

> ###### Dependencies:
> This project depends on `python3` and the following python libraries: `argparse` `requests` `imaplib`

### How to Use it
The application will execute the actions indicated by the entered flags and then _promptly_ exit. If any other actions are desired, run the program again using different flags.

<img src=".assets/images/BruteForcer.png" alt="BruteForcer.py on a Terminal Window" align="center"/>

##
> #### DISCLAIMER:
> **I do not condone the use of this software in any way that breaks the law, as it was made entirely for educational purposes.**
> 
> The software is offered as is, and **I do not** take responsibility for any kind of misuse.
##

<div align="center">

[![Back to the Top Badge](https://custom-icon-badges.demolab.com/badge/Back_to_the_Top-171515?logo=chevron-up)](#bruteforce-concept-demonstration)

</div>

# 📄 Criação da Tabela `BronzeYellow`
Este script unifica os dados mensais de viagens **Yellow Taxi** de diferentes meses (`Mes01` a `Mes05`)  
em uma única tabela `BronzeYellow`, utilizando a operação `UNION ALL` para manter **todos os registros**.

---

## 🎯 Objetivo
Consolidar dados de múltiplas tabelas mensais em uma única base para facilitar análises, relatórios e KPIs globais.

---


In [0]:
%sql
-- Seleciona o catálogo e schema de destino
USE CATALOG ifood_case;


USE SCHEMA nytaxi;

In [0]:
%sql
-- Cria ou substitui a tabela unificada BronzeYellow
CREATE OR REPLACE TABLE BronzeYellow AS

-- =========================================
-- 1️⃣ União de todas as tabelas mensais
--    - Tabelas de origem BronzeYellowMesXX já estão tratadas e higienizadas
--    - Remove duplicatas
-- =========================================
SELECT * FROM BronzeYellowMes01
UNION 
SELECT * FROM BronzeYellowMes02
UNION 
SELECT * FROM BronzeYellowMes03
UNION 
SELECT * FROM BronzeYellowMes04
UNION
SELECT * FROM BronzeYellowMes05;


# ✅ Validação da tabela `BronzeYellow`

Nesta versão:
- Cada regra de validação gera um elemento no array `lista_alertas`
- Valores `NULL` são removidos do array de mensagens
- `num_alertas`: quantidade de alertas por registro
- `alerta_validacao`: string única com alertas separados por `"; "`

---


In [0]:
%sql
WITH BronzeYellowValidate AS (
  SELECT
    Data_Fim_Viagem
    , Data_Inicio_Viagem
    , Dia_Fim_Viagem
    , Dia_Inicio_Viagem
    , Hora_Fim_Viagem
    , Hora_Inicio_Viagem
    , Mes_Fim_Viagem
    , Mes_Inicio_Viagem
    , passenger_count
    , tpep_dropoff_datetime
    , tpep_pickup_datetime
    , total_amount
    , vendorId

    -- =========================================
    -- ARRAY DE REGRAS DE VALIDAÇÃO
    -- Cada WHEN verdadeiro gera mensagem; falso gera NULL
    -- =========================================
    , array_remove(ARRAY(
      CASE WHEN Data_Fim_Viagem IS NULL
            OR (Data_Fim_Viagem < "2023-01-01" OR Data_Fim_Viagem > "2023-05-31")
            THEN 'data fim viagem inválido' END

      , CASE WHEN Data_Inicio_Viagem IS NULL
            OR (Data_Inicio_Viagem < "2023-01-01" OR Data_Inicio_Viagem > "2023-05-31")
            THEN 'data início viagem inválido' END

      , CASE WHEN Dia_Fim_Viagem IS NULL OR Dia_Fim_Viagem < 1 OR Dia_Fim_Viagem > 31
            THEN 'Dia_Fim_Viagem fora do intervalo' END

      , CASE WHEN Dia_Inicio_Viagem IS NULL OR Dia_Inicio_Viagem < 1 OR Dia_Inicio_Viagem > 31
            THEN 'Dia_Inicio_Viagem fora do intervalo' END

      , CASE WHEN Hora_Fim_Viagem IS NULL OR Hora_Fim_Viagem < '00' OR Hora_Fim_Viagem > '23'
            THEN 'Hora_Fim_Viagem fora do intervalo' END

      , CASE WHEN Hora_Inicio_Viagem IS NULL OR Hora_Inicio_Viagem < '00' OR Hora_Inicio_Viagem > '23'
            THEN 'Hora_Inicio_Viagem fora do intervalo' END

      , CASE WHEN Mes_Fim_Viagem IS NULL OR Mes_Fim_Viagem < 1 OR Mes_Fim_Viagem > 5
            THEN 'Mes_Fim_Viagem fora do intervalo' END

      , CASE WHEN Mes_Inicio_Viagem IS NULL OR Mes_Inicio_Viagem < 1 OR Mes_Inicio_Viagem > 5
            THEN 'Mes_Inicio_Viagem fora do intervalo' END

      , CASE WHEN passenger_count IS NULL OR passenger_count <= 0
            THEN 'passenger_count inválido' END

      , CASE WHEN total_amount IS NULL OR total_amount <= 0
            THEN 'total_amount inválido' END

      , CASE WHEN vendorId IS NULL OR (vendorId NOT IN (1, 2, 6, 7))
            THEN 'vendorId inválido' END
    ), NULL) AS lista_alertas
  FROM BronzeYellow
)

SELECT
    *
    , SIZE(lista_alertas)             AS  num_alertas       -- Quantidade de alertas
    , CONCAT_WS('; ', lista_alertas)  AS  alerta_validacao  -- String com alertas separados
FROM 
  BronzeYellowValidate
WHERE 
  SIZE(lista_alertas) > 0; -- Retorna somente registros com alertas
